In [1]:
import pandas as pd
from keras.layers import LSTM, Input, Bidirectional, TimeDistributed, Dropout, Dense, Activation
from keras.models import Model
from keras.utils.np_utils import to_categorical
import tensorflow as tf
import numpy as np
import os

Using TensorFlow backend.


## Model

In [2]:
def get_model(input_shape, output_shape, lr = 0.00001):
    inputs = Input(shape=input_shape)
    X = Dropout(0.5)(inputs)
    X = Dense(4096, activation='relu')(X)
    X = Dropout(0.5)(X)
    X = Dense(2048, activation='relu')(X)
    X = Dense(2048, activation='relu')(X)
    X = Bidirectional(LSTM(1024, return_sequences=True))(X)
    outputs = TimeDistributed(Dense(output_shape, activation='softmax'))(X)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=tf.train.AdamOptimizer(lr),
        loss='categorical_crossentropy',
        metrics=['categorical_accuracy'],
    )
    
    return model

## Data

In [3]:
def generator(path, mode, num_batches):
    counter = 0
    while True:
        if counter >= num_batches:
            counter = 0

        features = np.load(path + 'features_' + mode + '_' + str(counter) + '.npy')
        classes = np.load(path + 'classes_' + mode + '_' + str(counter) + '.npy')
        
        counter += 1
        yield features, classes

## Training

In [4]:
model = get_model((2, 2048), 167)

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2, 2048)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2, 2048)           0         
_________________________________________________________________
dense_1 (Dense)              (None, 2, 4096)           8392704   
_________________________________________________________________
dropout_2 (Dropout)          (None, 2, 4096)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 2, 2048)           8390656   
_________________________________________________________________
dense_3 (Dense)              (None, 2, 2048)           4196352   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 2, 2048)           25174016  
__________

In [ ]:
model.fit_generator(
    generator('./features/lstm/2_steps/', 'train', 101),
    steps_per_epoch=101,
    epochs=50,
    validation_data=generator('./features/lstm/2_steps/', 'dev', 26),
    validation_steps=26,
)

Epoch 1/50
101/101 [==============================] - 133s 1s/step - loss: 4.7109 - categorical_accuracy: 0.0571 - val_loss: 4.5204 - val_categorical_accuracy: 0.0586
Epoch 2/50
101/101 [==============================] - 146s 1s/step - loss: 4.4118 - categorical_accuracy: 0.0644 - val_loss: 4.2911 - val_categorical_accuracy: 0.0815
Epoch 3/50
 98/101 [============================>.] - ETA: 3s - loss: 4.2070 - categorical_accuracy: 0.0831